# Face Recognition w/ 3D Landmarks

Melisa Mete, 150200316

Öykü Eren, 150200326

Bora Boyacıoğlu, 150200310

## Step 1: Data Preparation and Preprocessing

Import necessary libraries.

In [1]:
%pip install mediapipe dlib opencv-python numpy plotly scipy --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
import warnings

import dlib

from mediapipe.tasks import python
from mediapipe.tasks.python import vision

from utils.images import Images
from utils.utils import *

2024-05-21 16:06:12.520944: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
# Disable UserWarning.
warnings.filterwarnings("ignore", category = UserWarning)

### 1.1. Preprocessing

In [5]:
# Load all the images.
data_dir = 'data/lfw-deepfunneled/'
images = Images()
images.read(data_dir)

Reading images... 100.00%


In [6]:
# Apply preprocessing to all images.
images.images = images.apply(preprocess, target_size=(224, 224))

Applying preprocess... 100.00%


### 1.2. Landmark Extraction

In [7]:
# Define the face mesh model.
base_options = python.BaseOptions(model_asset_path='model/face_landmarker_v2_with_blendshapes.task')
options = vision.FaceLandmarkerOptions(base_options=base_options,
                                       output_face_blendshapes=True,
                                       output_facial_transformation_matrixes=True,
                                       num_faces=1)
detector_3d = vision.FaceLandmarker.create_from_options(options)

I0000 00:00:1716296874.164152 1232557 gl_context.cc:357] GL version: 2.1 (2.1 INTEL-22.5.10), renderer: Intel(R) Iris(TM) Plus Graphics 645
W0000 00:00:1716296874.197405 1232557 face_landmarker_graph.cc:174] Sets FaceBlendshapesGraph acceleration to xnnpack by default.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1716296874.255185 1234532 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1716296874.288196 1234537 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [8]:
# Apply the face mesh model to all images.
images.landmarks = images.apply(extract_landmarks, detector=detector_3d)

Applying extract_landmarks... 100.00%


### 1.3. Train-Test Split

In [9]:
# Create the NumPy arrays.
X1 = [np.array(photos) for photos in list(images.images.values())]
X2 = [np.array(landmarks) for landmarks in list(images.landmarks.values())]

# Create label instances.
vocab, y = np.unique(list(images.images.keys()), return_inverse=True)

In [10]:
# Train-Test Split.
splits = train_test_split(X1, X2, y)
X1_train, X1_test, X2_train, X2_test, y_train, y_test = splits

In [11]:
# Reshapes and NumPy arrays.
# Train set.
X1_train = np.array([i.reshape(224, 224, 3) for i in X1_train])
X2_train = np.array([i.reshape(478, 3) for i in X2_train])
y_train = np.array(y_train)

# Test set.
X1_test = np.array([i.reshape(224, 224, 3) for i in X1_test])
X2_test = np.array([i.reshape(478, 3) for i in X2_test])
y_test = np.array(y_test)

### 1.4. Face Realignment

#### 1.4.1. Landmarks

In [12]:
# Merge the train and test for X2.
X2 = np.concatenate((X2_train, X2_test), axis=0)

# Get the stabilized indices.
stable_indices_3d = stabilized_lm(X2, N=10)

In [13]:
# Realign the X2 train set.
X2_train = realign_lm(X2_train, stable_indices_3d)

# Realign the X2 test set.
X2_test = realign_lm(X2_test, stable_indices_3d)

#### 1.4.2. Images

In [14]:
# Load the face detector and shape predictor from dlib.
detector_2d = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('model/shape_predictor_68_face_landmarks.dat')

In [15]:
# Exract 2D landmarks for faces.
X1_train_lm = extract_2d_lm(X1_train, detector_2d, predictor)
X1_test_lm = extract_2d_lm(X1_test, detector_2d, predictor)

Extracting landmarks... 0.02%

Extracting landmarks... 100.00%
Extracting landmarks... 100.00%


In [16]:
# Merge the train and test for X1 landmarks.
X1_lm = np.concatenate((X1_train_lm, X1_test_lm), axis=0)

# Get the stabilized indices.
stable_indices_2d = stabilized_lm(X1_lm, N=10)

In [17]:
# Realign the X1 train set.
X1_train = realign_img(X1_train_lm, X1_train, stable_indices_2d)

# Realign the X1 test set.
X1_test = realign_img(X1_test_lm, X1_test, stable_indices_2d)

Realigning images... 100.00%
Realigning images... 100.00%


### 1.5. Saving the Data

In [18]:
# Print the shapes.
print(f"{X1_train.shape = }",
      f"{X2_train.shape = }",
      f"{y_train.shape = }",
      f"{X1_test.shape = }",
      f"{X2_test.shape = }",
      f"{y_test.shape = }",
      sep='\n')

X1_train.shape = (9964, 224, 224, 3)
X2_train.shape = (9964, 478, 3)
y_train.shape = (9964,)
X1_test.shape = (3269, 224, 224, 3)
X2_test.shape = (3269, 478, 3)
y_test.shape = (3269,)


In [19]:
# Save the NumPy arrays.
np.save('data/arr/X1_train.npy', X1_train)
np.save('data/arr/X2_train.npy', X2_train)
np.save('data/arr/y_train.npy', y_train)
np.save('data/arr/X1_test.npy', X1_test)
np.save('data/arr/X2_test.npy', X2_test)
np.save('data/arr/y_test.npy', y_test)

In [30]:
# Get the index of 'Aldo_Paredes' in the vocabulary.
np.where(vocab == 'Aldo_Paredes')[0][0]

117

Show an example plot.

In [23]:
plot_lm(X2_train[-1], stable_indices_3d)